In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from geo_dist_prep.data import GEONAMES_DB
from geo_dist_prep.schemas.geoname import GeoName
from geo_dist_prep.schemas.geoname_pair import GeoNamePair
from geo_dist_prep.schemas.job import GeoNamePairJob

engine = create_engine(f"sqlite:///../{GEONAMES_DB}")

Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
import pandas as pd

from sqlalchemy.sql import and_, select
from sqlalchemy import func, bindparam, Integer, text, Float, union_all
import matplotlib.pyplot as plt

from geo_dist_prep.schemas.helpers import *
from geo_dist_prep.schemas.job import Job, GeoNamePairJob



southport = 26700924
lon, lat = -3.004175, 53.647599

#query = session.query(GeoName, GeoNamePair).join(GeoNamePair, GeoName.osm_id == GeoNamePair.start_id).filter(GeoName.score > 32)
query = session.query(GeoName).filter(GeoName.country_code == 'at')
q = query.statement.compile()

df = pd.read_sql_query(q, session.bind)
print(len(df))
plt.scatter(df["lon"], df["lat"], s=1)

In [ ]:
query =union_all(
    session.query(GeoName).filter(GeoName.country_code == "uk"),
    session.query(GeoName).filter(GeoName.country_code == "ie")
)

q = query.compile()
df = pd.read_sql_query(q, session.bind)

df
